In [1]:
from bs4 import BeautifulSoup
import urllib.request
import selenium
from selenium import webdriver
from urllib.request import urlopen
from selenium.webdriver.common.by import By
import re
import time
from tqdm import tqdm_notebook
from selenium.webdriver.common.keys import Keys
import pandas as pd

In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [3]:
def extract_review(place_id, driver):
#    while True:
#         try:
#             if driver.find_element(By.CSS_SELECTOR, '#marticle > div.cont_evaluation > div.evaluation_review > a').text == '후기 더보기':
#                 driver.find_element(By.CSS_SELECTOR, '#marticle > div.cont_evaluation > div.evaluation_review > a').click()
#             else:
#                 break
#         except:
#             break

#mArticle > div.cont_evaluation > div.evaluation_review > a > span.txt_more
#mArticle > div.cont_evaluation > div.evaluation_review > a
#mArticle > div.cont_evaluation > div.evaluation_review > a
    time.sleep(1)
    while True:
        try:
            if driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.evaluation_review > a').text == '후기 더보기':
                driver.execute_script("arguments[0].click();", driver.find_element(By.CSS_SELECTOR, '#mArticle > div.cont_evaluation > div.evaluation_review > a'))
            else:
                break
        except:
            break
        time.sleep(1)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        review_lists = soup.find('ul', {'class':'list_evaluation'}).find_all('li')
    except:
        return ''
    
    p_name = []
    u_name = []
    u_rating = []
    u_comment = []
    
    for review_list in review_lists:
        try:
            user_id = review_list.find('a', {'class':'link_user'}).text
        except:
            user_id = ''
            continue
        
        try:
            value = review_list.find('div', {'class':'star_info'}).find('div').find('span').find('span')['style']
            value = value.replace('width:', '')
            value = value.replace('%;', '')
            user_rating = int(value)/20
        except:
            user_rating = ''
            continue
        
        try:
            user_comment = review_list.find('p', {'class':'txt_comment'}).find('span').text
        except:
            user_comment = ''
            continue
        p_name.append(place_id)
        u_name.append(user_id)
        u_rating.append(user_rating)
        u_comment.append(user_comment)
        
        print('place_id:', place_id)
        print('user_id:', user_id)
        print('score:', user_rating)
        print('comment:', user_comment)
    
    review_df = pd.DataFrame({
        'place_id':p_name,
        'user_id':u_name,
        'score':u_rating,
        'comment':u_comment
    },)

    review_df.to_csv(f'./review/{place_id}.csv', index=False, encoding='utf-8-sig')
    # return f'./kakao_reviews/{place_key}/{place_id}.csv'



In [4]:
url = 'https://map.kakao.com/'
chromedriver_dir = r'./chromedriver.exe'
driver = webdriver.Chrome(chromedriver_dir)
driver.get(url)
time.sleep(1)

inputbox = driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]')
driver.execute_script("arguments[0].click();", inputbox)
inputbox.send_keys('울산 음식점')
inputbox.send_keys(Keys.ENTER)
#info = driver.find_element(By.CLASS_NAME, 'Info')
time.sleep(1)



print(driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div[1]/div[7]/div[5]/div[1]/ol/li[2]/a').text)
driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div[1]/div[7]/div[5]/div[1]/ol/li[2]/a'))


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_8\3139736520.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver_dir)


인기도순


In [5]:
ul = driver.find_element(By.CSS_SELECTOR, '#info\.search\.place\.list')
driver.execute_script("arguments[0].click();", ul)
lis = ul.find_elements(By.TAG_NAME, 'li')
len(lis)

15

In [6]:
# //*[@id="info.search.place.list"]/li[1]
# //*[@id="info.search.place.list"]/li[1]/div[4]/span[1]/a
title = []
address = []
url_lst = []
rate = []
tags = []
#info\.search\.place\.list > li:nth-child(1) > div.info_item > div.contact.clickArea > a.moreview
# //*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]
#info\.search\.place\.list
page_num = 1
while page_num != 21:
    time.sleep(1)
    ul = driver.find_element(By.XPATH, '//*[@id="info.search.place.list"]')
    driver.execute_script("arguments[0].click();", ul)
    time.sleep(1)
    lis = ul.find_elements(By.TAG_NAME, 'li')
    print(len(lis))
    idx = page_num % 5

    for li in lis:
        print(li)
        time.sleep(1)
        # driver.execute_script("arguments[0].click();", li.find_element(By.XPATH, '//div[4]/span[1]/a'))
        # li.find_element(By.CSS_SELECTOR, 'div.info_item > div.contact.clickArea > a.moreview').send_keys(Keys.ENTER)
        # //*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]
        # //*[@id="info.search.place.list"]/li[1]/div[5]/div[4]/a[1]
        # //*[@id="info.search.place.list"]/li[2]/div[5]/div[4]/a[1]
        driver.execute_script("arguments[0].click();", li.find_element(By.CSS_SELECTOR, 'div.info_item > div.contact.clickArea > a.moreview'))
        # driver.execute_script("arguments[0].click();", li.find_element(By.XPATH, '//div[5]/div[4]/a[1]'))
        
        driver.switch_to.window(window_name=driver.window_handles[-1])
        
        print(driver.current_url)
        time.sleep(2)
        p_id = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2').text
        extract_review(place_id=p_id, driver=driver)
        driver.switch_to.window(window_name=driver.window_handles[-1])
        
        
        driver.close()
        driver.switch_to.window(window_name=driver.window_handles[0])
        # driver.execute_script("arguments[0].click();", li.find_element(By.XPATH, ' #info\.search\.place\.list > li.PlaceItem.clickArea.PlaceItem-ACTIVE > div.rating.clickArea > span.score > a'))
        
    if idx == 0:
        driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH, '//*[@id="info.search.page.next"]'))
    else:
        driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH, '//*[@id="info.search.page.no' + str(idx + 1) +'"]'))
        
    page_num += 1

driver.quit()

15
<selenium.webdriver.remote.webelement.WebElement (session="5bff9fb76908859bd50f91fec373c757", element="f9a3bbfd-f907-46fe-8339-de904732a617")>
https://place.map.kakao.com/16810283
place_id: 언양기와집불고기
user_id: 이무지유느
score: 3.0
comment: 정말 무시무시한 웨이팅.. 식사시간 좀 지나서 가고 그런거랑 상관없습니다근데 불고기는 기똥차게 맛있어요달지만 맛있는 단맛육회도 맛있어용
place_id: 언양기와집불고기
user_id: oh
score: 3.0
comment: 비싸고 맛은 괜찮은편인데 장사가 잘돼서 그런지 오픈런했는데 빨리 먹고 나가라고 압박이 좀 있네요또 갈지는 생각 좀 해봐야겠습니다
place_id: 언양기와집불고기
user_id: 치타
score: 5.0
comment: 하.. 존맛탱입니다. 울산 갈 때마다 꼭 갑니다.가끔 이거 먹으러 울산까지 갈 때도 있습니다.
place_id: 언양기와집불고기
user_id: 후라이
score: 5.0
comment: 갓벽
place_id: 언양기와집불고기
user_id: 가나다라마바사
score: 4.0
comment: 언양불고기 먹고싶을 때 가는곳추천드립니다
place_id: 언양기와집불고기
user_id: 또몽이
score: 4.0
comment: 맛있고 웨이팅 좀 길어서 오픈런 추천합니다
place_id: 언양기와집불고기
user_id: 하하하
score: 5.0
comment: 오랜만에 방문했는데 여전히 맛있고 친절하시네요^^ 잘 먹고 갑니다. 언양불고기3 물막국수1비빔막국수1된장찌개1공기밥2: 완벽클리어ㅎㅎ후식으로 나오는 식혜도 너무 맛나요~
place_id: 언양기와집불고기
user_id: leesj
score: 5.0
comment: 이집은 불고기가 아니라 생고기가 맛있음 육회나 불고기는 단맛이 좀 있음. 불고기 식감은 

In [26]:
print(title)
print(address)
print(url_lst)
print(rate)
print(tags)

['방이편백육분삼십 울산태화점', '요리사의집 느티나무', '애월목수', '언양원조불고기', '발리정원', '미강', '환장라멘', '백림', '명촌순두부보쌈', '착한물고기', '생식본능', '스페이스오드', '금강산삼계탕', '라오', '진하꽃게전문점', '육일횟집', '콩나물교실해장국 울산본점', '스타벅스 울산성안점', '백년본가김치찜', '청기와숯불갈비', '마운트', '원터가든 명촌점', '호재래', '고반식당 삼산점', '등대갈비', '모노컬러', '토지보리밥', '여행가다', '카페 베르츠율리', '코모레비', '대가야삼계탕 울산점', '달동생아구', '랑콩뜨레 울산직영점', '산동만두', '떡나래', '보리꽃', '이모집', '리플렉티브', '풍로옥', '컴포트', '해마지', '숙성도마 울산삼산점', '이어도', '거부돌곱창', '골든드롭', '봉계한우마을', '하와이새우트럭 간절곶점', '태번159', '복산육관', '어니스트케이프', '신가네밀면', '와사비', '라멘집입니다 울산무거점', '숙성창고', '공주분식', '카페발코니', '노아레스토랑', '사랑채', '교동식당', '미락한우명가', '홍익돈까스 울산동구일산점', '탕화쿵푸마라탕 삼산점', '진호관', '삿뽀로 울산점', '함경면옥', '소진담', '진주횟집', '수림복국', '더바베큐캠프', '밀면전문점', '대한상회 울산삼산점', '더짬뽕', '24시언양돼지국밥', '랑콩뜨레 달동점', '카페이소', '장군참치', '유객주', '브루센트', '대구짬뽕정', '화공', '동해반점', '정자참가자미', '언양불고기식당', '금화불고기', '스컹크웍스 신정점', '서생집', '고집센그집갈비탕 울산본점', '함양집', '무라카데칼국수', '샤브나인', '용궁해물탕', '퐁당퐁당 본점', '복산돈까스', '화봄', '뜨락전통손칼국수', '안주일체', '게스트로', '홍이', '경아식당', '리코파스토', '곱간 울산본점', '이너리트', '은린', '구룡', '스타벅스 울산간절곶점'

In [27]:
len(title)

150

In [28]:

df = pd.DataFrame({'title': title, 'rate': rate, 'tags': tags, 'address': address, 'url': url_lst})
df

,title,rate,tags,address,url
0,방이편백육분삼십 울산태화점,3.8,"육류,고기",울산 중구 신기9길 62 1층 (우)44456,https://place.map.kakao.com/1498733640
1,요리사의집 느티나무,4.1,삼계탕,울산 북구 진장1길 30 1층 (우)44251,https://place.map.kakao.com/24710626
2,애월목수,4.0,"호프,요리주점",울산 남구 달삼로 58 1층 (우)44721,https://place.map.kakao.com/1281670667
3,언양원조불고기,4.2,"불고기,두루치기",울산 울주군 언양읍 남천로 200 (우)44945,https://place.map.kakao.com/20955677
4,발리정원,3.7,카페,울산 울주군 온양읍 선양골길 105-16 (우)44976,https://place.map.kakao.com/127518725
...,...,...,...,...,...
145,도드람마을,3.5,"육류,고기",울산 북구 진장로 85 (우)44250,https://place.map.kakao.com/26345203
146,배내통하우스,5.0,한식,울산 울주군 상북면 배내로 1447 (우)44909,https://place.map.kakao.com/10393475
147,송가정횟집,3.6,회,울산 남구 대학로 3-1 (우)44616,https://place.map.kakao.com/27140411
148,스타벅스 울산삼산로점,1.7,스타벅스,울산 남구 삼산로278번길 8 (우)44719,https://place.map.kakao.com/26506097


In [29]:
df.to_csv('ulsan_titelist.csv', index = False, encoding='utf-8-sig')